In [1]:
# Install gdown
!pip install gdown

# Import gdown
import gdown

# Folder ID
folder_id = '1i9mph0-by_Rm2J8iKtx9Hf-iaDJvMETU'

# Download the folder
gdown.download_folder(f'https://drive.google.com/drive/folders/{folder_id}', output='/content/my_data')

# List files in the downloaded folder
import os
folder_path = '/content/my_data'
print(os.listdir(folder_path))



Retrieving folder contents


Processing file 1UeFNeNppWNNsbp-2ZfwfWHgeOhtQalBX added_tokens.json
Processing file 11C_wpxWmvaaaX--rut85tKGUa8pMvCjz config.json
Processing file 1oP1MeEp5UfEOIraqvmeaweflPCucACRI generation_config.json
Processing file 1wqhQ9oc_E4g49UsgFvn5DbVkQnyW993T merges.txt
Processing file 1LQdIED_A8WH_0o5W-QmIChOWVrJ1R-Ia model.safetensors
Processing file 1TGKaetXB3TCnciRL0Ig_EgTBv22LthEu normalizer.json
Processing file 14RKL4Rm6OD3zjqFCxGD-wsDPoj9osWaS preprocessor_config.json
Processing file 1iOYjc3u41zKk0SkcK4iigrXBL4aeTBut special_tokens_map.json
Processing file 133gdidBz9W9slx0A8GayZxDAIn8yQMZ_ tokenizer_config.json
Processing file 1hTUxudqwTIePwm9qT1R-J0_A4-Q8LpBG vocab.json


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1UeFNeNppWNNsbp-2ZfwfWHgeOhtQalBX
To: /content/my_data/added_tokens.json
100%|██████████| 2.22k/2.22k [00:00<00:00, 3.33MB/s]
Downloading...
From: https://drive.google.com/uc?id=11C_wpxWmvaaaX--rut85tKGUa8pMvCjz
To: /content/my_data/config.json
100%|██████████| 1.33k/1.33k [00:00<00:00, 2.80MB/s]
Downloading...
From: https://drive.google.com/uc?id=1oP1MeEp5UfEOIraqvmeaweflPCucACRI
To: /content/my_data/generation_config.json
100%|██████████| 290/290 [00:00<00:00, 447kB/s]
Downloading...
From: https://drive.google.com/uc?id=1wqhQ9oc_E4g49UsgFvn5DbVkQnyW993T
To: /content/my_data/merges.txt
100%|██████████| 544k/544k [00:00<00:00, 85.8MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1LQdIED_A8WH_0o5W-QmIChOWVrJ1R-Ia
From (redirected): https://drive.google.com/uc?id=1LQdIED_A8WH_0o5W-QmIChOWVrJ1R-Ia&confirm=t&uuid=1d

['preprocessor_config.json', 'tokenizer_config.json', 'model.safetensors', 'special_tokens_map.json', 'generation_config.json', 'normalizer.json', 'config.json', 'merges.txt', 'vocab.json', 'added_tokens.json']



Download completed


In [4]:
import torch
import torchaudio
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq

# Paths
model_path = "/content/my_data"
audio_path = "/content/33.mp3"

# Load model and processor
processor = AutoProcessor.from_pretrained(model_path)
model = AutoModelForSpeechSeq2Seq.from_pretrained(model_path).to("cpu")

# Load and preprocess audio
speech_array, sampling_rate = torchaudio.load(audio_path)
if sampling_rate != 16000:
    resampler = torchaudio.transforms.Resample(orig_freq=sampling_rate, new_freq=16000)
    speech_array = resampler(speech_array)

# Mono channel expected
if speech_array.shape[0] > 1:
    speech_array = speech_array.mean(dim=0, keepdim=True)

# Remove channel dimension
input_features = processor.feature_extractor(speech_array.squeeze().numpy(), sampling_rate=16000, return_tensors="pt").input_features

# Generate
with torch.no_grad():
    generated_ids = model.generate(input_features)

# Decode
transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
print("📝 Transcribed Text:", transcription)


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


📝 Transcribed Text: إِنَّ اللَّهَ لَا يَسْتَحْيِي أَنْ يَضْرِبَ مَثَلًا مَا بَعُوضَةً فَمَا فَوْقَهَا فَأَمَّا الَّذِينَ آمَنُوا فَيَعْلَمُونَ أَنَّهُ الْحَقُّ مِنْ رَبِّهِمْ وَأَمَّا الَّذِينَ كَفَرُوا فَيَقُولُونَ مَاذَا أَرَادَ اللَّهُ بِهَذَا مَثَلًا


In [1]:
!pip install flask flask-ngrok torchaudio transformers

In [5]:
!pip install pyngrok
from flask import Flask, request, jsonify
from pyngrok import ngrok
import torch
import torchaudio
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq
import os

# Setup
app = Flask(__name__)

# 🔥 Load model and processor once
MODEL_PATH = "/content/my_data"  # Your model folder path
processor = AutoProcessor.from_pretrained(MODEL_PATH)
model = AutoModelForSpeechSeq2Seq.from_pretrained(MODEL_PATH).to("cpu")

# 🎙️ Function to transcribe audio
def transcribe_audio(audio_path):
    waveform, sample_rate = torchaudio.load(audio_path)

    # Convert stereo to mono if needed
    if waveform.shape[0] > 1:
        waveform = waveform.mean(dim=0, keepdim=True)

    # Resample if not 16kHz
    if sample_rate != 16000:
        resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)
        waveform = resampler(waveform)

    input_features = processor.feature_extractor(
        waveform.squeeze().numpy(), sampling_rate=16000, return_tensors="pt"
    ).input_features

    with torch.no_grad():
        generated_ids = model.generate(input_features)

    transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return transcription

# 🔥 Flask endpoint
@app.route('/transcribe', methods=['POST'])
def transcribe():
    if 'audio' not in request.files:
        return jsonify({'error': 'No audio file uploaded'}), 400

    audio_file = request.files['audio']
    file_path = os.path.join("/content", audio_file.filename)
    audio_file.save(file_path)

    try:
        text = transcribe_audio(file_path)
        return jsonify({'transcription': text})
    except Exception as e:
        return jsonify({'error': str(e)}), 500
    finally:
        os.remove(file_path)

# Set your ngrok authtoken here
# Replace "YOUR_AUTHTOKEN" with the actual token from your ngrok dashboard
ngrok.set_auth_token("2wwoXpT4cJkm5oFI6tbeH3zYJPR_871PxxvpSbr1aXdwxGY3e")

# 🚀 Start the API and ngrok tunnel
public_url = ngrok.connect(5000)
print("🚀 Your public ngrok URL:", public_url)

# 🔥 Run Flask
app.run()

🚀 Your public ngrok URL: NgrokTunnel: "https://e975-35-233-211-24.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [21/May/2025 20:11:33] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [21/May/2025 20:11:33] "GET /favicon.ico HTTP/1.1" 404 -
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
INFO:werkzeug:127.0.0.1 - - [21/May/2025 20:15:12] "POST /transcribe HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/May/2025 20:15:55] "POST /transcribe HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/May/2025 20:19:29] "POST /transcribe HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/May/2025 20:20:49] "POST /transcribe HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/May/2025 20:21:49] "GET /transcribe'

In [ ]:
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, TFAutoModelForSpeechSeq2Seq
import tensorflow as tf

# Load the model and processor
model_path = "/content/my_data"  # Path to your model directory
processor = AutoProcessor.from_pretrained(model_path)
model = AutoModelForSpeechSeq2Seq.from_pretrained(model_path)

# Convert PyTorch model to TensorFlow
tf_model = TFAutoModelForSpeechSeq2Seq.from_pretrained(model_path, from_pt=True)

# Save the TensorFlow model
tf_model.save("tf_model", save_format="tf")

All PyTorch model weights were used when initializing TFWhisperForConditionalGeneration.

All the weights of TFWhisperForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFWhisperForConditionalGeneration for predictions without further training.


In [ ]:
import tensorflow as tf

# Load the TensorFlow model
tf_model = tf.saved_model.load("tf_model")

# Convert to TFLite
converter = tf.lite.TFLiteConverter.from_saved_model("tf_model")
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,
    tf.lite.OpsSet.SELECT_TF_OPS  # Needed for transformer ops
]
converter.experimental_new_converter = True
tflite_model = converter.convert()

# Save the TFLite model
with open("model.tflite", "wb") as f:
    f.write(tflite_model)

In [ ]:
converter.optimizations = [tf.lite.Optimize.DEFAULT]  # Quantization to reduce size
tflite_model = converter.convert()
with open("model.tflite", "wb") as f:
    f.write(tflite_model)

In [ ]:
import tensorflow.lite as tflite
import numpy as np
import librosa
from transformers import AutoProcessor

# Load the processor for tokenization/decoding
model_path = "/content/my_data"
processor = AutoProcessor.from_pretrained(model_path)

# Load Arabic audio (16 kHz, mono)
audio_path = "/content/001 Al-Fatihah alfath.wav"  # Replace with your Arabic WAV file
audio, sr = librosa.load(audio_path, sr=16000, mono=True)

# Whisper expects 30-second chunks; pad or truncate
max_length = 30 * 16000  # 30 seconds at 16 kHz
if len(audio) > max_length:
    audio = audio[:max_length]
else:
    audio = np.pad(audio, (0, max_length - len(audio)), mode="constant")

# Normalize audio to [-1, 1]
audio = audio / np.max(np.abs(audio))

# Prepare audio input (shape: [1, 480000])
audio_input = audio.reshape(1, -1).astype(np.float32)

# Load TFLite model
interpreter = tflite.Interpreter(model_path="/content/model.tflite")
interpreter.allocate_tensors()

# Get input/output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Print all input details to identify the correct tensors
print("All Input Details:")
for i, detail in enumerate(input_details):
    print(f"Input {i}:")
    print(f"  Name: {detail['name']}")
    print(f"  Shape: {detail['shape']}")
    print(f"  Type: {detail['dtype']}")

print("\nOutput Details:")
for detail in output_details:
    print(f"  Shape: {detail['shape']}")
    print(f"  Type: {detail['dtype']}")

# Prepare inputs (based on the error, we need to handle multiple inputs)
decoder_input_ids = np.array([[50258]], dtype=np.int32)  # Start token for Whisper
transcription_ids = []

# Autoregressive decoding loop
max_output_length = 448  # Max transcription length
for _ in range(max_output_length):
    # Update attention mask to match the current sequence length
    attention_mask = np.ones((1, decoder_input_ids.shape[1]), dtype=np.int32)

    # Set inputs
    for detail in input_details:
        if "input_features" in detail["name"]:
            interpreter.set_tensor(detail["index"], audio_input)
        elif "attention_mask" in detail["name"]:
            interpreter.set_tensor(detail["index"], attention_mask)
        elif "input_ids" in detail["name"]:
            interpreter.set_tensor(detail["index"], decoder_input_ids)

    # Run inference
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]["index"])

    # Get the predicted token
    predicted_token = np.argmax(output_data[0, -1, :])  # Last token
    transcription_ids.append(predicted_token)

    # Stop if end token is generated
    if predicted_token == 50256:  # Whisper end token
        break

    # Append predicted token for the next iteration
    decoder_input_ids = np.append(decoder_input_ids, [[predicted_token]], axis=1)

# Decode the token IDs to Arabic text
transcription = processor.batch_decode([transcription_ids])[0]
print("Arabic Transcription:", transcription)

All Input Details:
Input 0:
  Name: serving_default_decoder_attention_mask:0
  Shape: [1 1]
  Type: <class 'numpy.int32'>
Input 1:
  Name: serving_default_decoder_input_ids:0
  Shape: [1 1]
  Type: <class 'numpy.int32'>
Input 2:
  Name: serving_default_input_features:0
  Shape: [ 1 80  1]
  Type: <class 'numpy.float32'>

Output Details:
  Shape: [    1     1 51865]
  Type: <class 'numpy.float32'>
  Shape: [   1 1500  512]
  Type: <class 'numpy.float32'>


ValueError: Cannot set tensor: Dimension mismatch. Got 2 but expected 3 for input 2.

In [ ]:
import tensorflow.lite as tflite
import numpy as np
import librosa
from transformers import AutoProcessor

# Load the processor for tokenization and feature extraction
model_path = "/content/my_data"  # Path to your model directory
processor = AutoProcessor.from_pretrained(model_path)

# Load Arabic audio (16 kHz, mono)
audio_path = "/content/001 Al-Fatihah alfath.wav"  # Replace with your Arabic WAV file
audio, sr = librosa.load(audio_path, sr=16000, mono=True)

# Whisper expects 30-second chunks; pad or truncate
max_length = 30 * 16000  # 30 seconds at 16 kHz
if len(audio) > max_length:
    audio = audio[:max_length]
else:
    audio = np.pad(audio, (0, max_length - len(audio)), mode="constant")

# Normalize audio to [-1, 1]
audio = audio / np.max(np.abs(audio))

# Convert audio to log-Mel spectrogram features using the processor
# Whisper expects features of shape [1, 80, 3000] typically, but TFLite model expects [1, 80, 1]
inputs = processor(audio, sampling_rate=16000, return_tensors="np")
audio_input = inputs["input_features"]  # Shape: [1, 80, 3000]

# Since the TFLite model expects [1, 80, 1], we need to adjust (this may indicate a conversion issue)
# For now, let's take a slice or average to match the expected shape
audio_input = audio_input[:, :, :1]  # Take first time step: [1, 80, 1]

# Load TFLite model
interpreter = tflite.Interpreter(model_path="/content/model.tflite")
interpreter.allocate_tensors()

# Get input/output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Print all input details
print("All Input Details:")
for i, detail in enumerate(input_details):
    print(f"Input {i}:")
    print(f"  Name: {detail['name']}")
    print(f"  Shape: {detail['shape']}")
    print(f"  Type: {detail['dtype']}")

print("\nOutput Details:")
for detail in output_details:
    print(f"  Shape: {detail['shape']}")
    print(f"  Type: {detail['dtype']}")

# Prepare inputs
decoder_input_ids = np.array([[50258]], dtype=np.int32)  # Start token for Whisper
transcription_ids = []

# Autoregressive decoding loop
max_output_length = 448  # Max transcription length
for _ in range(max_output_length):
    # Update attention mask to match the current sequence length
    attention_mask = np.ones((1, decoder_input_ids.shape[1]), dtype=np.int32)

    # Set inputs
    for detail in input_details:
        if "input_features" in detail["name"]:
            interpreter.set_tensor(detail["index"], audio_input)
        elif "attention_mask" in detail["name"]:
            interpreter.set_tensor(detail["index"], attention_mask)
        elif "input_ids" in detail["name"]:
            interpreter.set_tensor(detail["index"], decoder_input_ids)

    # Run inference
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]["index"])

    # Get the predicted token
    predicted_token = np.argmax(output_data[0, -1, :])  # Last token
    transcription_ids.append(predicted_token)

    # Stop if end token is generated
    if predicted_token == 50256:  # Whisper end token
        break

    # Append predicted token for the next iteration
    decoder_input_ids = np.append(decoder_input_ids, [[predicted_token]], axis=1)

# Decode the token IDs to Arabic text
transcription = processor.batch_decode([transcription_ids])[0]
print("Arabic Transcription:", transcription)

All Input Details:
Input 0:
  Name: serving_default_decoder_attention_mask:0
  Shape: [1 1]
  Type: <class 'numpy.int32'>
Input 1:
  Name: serving_default_decoder_input_ids:0
  Shape: [1 1]
  Type: <class 'numpy.int32'>
Input 2:
  Name: serving_default_input_features:0
  Shape: [ 1 80  1]
  Type: <class 'numpy.float32'>

Output Details:
  Shape: [    1     1 51865]
  Type: <class 'numpy.float32'>
  Shape: [   1 1500  512]
  Type: <class 'numpy.float32'>


ValueError: Cannot set tensor: Dimension mismatch. Got 2 but expected 1 for dimension 1 of input 0.

In [ ]:
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, TFAutoModelForSpeechSeq2Seq
import tensorflow as tf
import numpy as np

# Load the model and processor
model_path = "my_model"  # Replace with your local model directory
processor = AutoProcessor.from_pretrained(model_path)
model = AutoModelForSpeechSeq2Seq.from_pretrained(model_path)

# Convert to TensorFlow
tf_model = TFAutoModelForSpeechSeq2Seq.from_pretrained(model_path, from_pt=True)

# Create a concrete function with dynamic shapes
@tf.function(input_signature=[
    tf.TensorSpec([1, 80, 3000], tf.float32, name="input_features"),
    tf.TensorSpec([1, None], tf.int32, name="decoder_input_ids"),
    tf.TensorSpec([1, None], tf.int32, name="decoder_attention_mask"),
])
def serving_fn(input_features, decoder_input_ids, decoder_attention_mask):
    return tf_model(
        input_features=input_features,
        decoder_input_ids=decoder_input_ids,
        attention_mask=decoder_attention_mask,
        training=False
    )

# Save the model with the signature
tf.saved_model.save(tf_model, "tf_model", signatures={"serving_default": serving_fn})

# Convert to TFLite
converter = tf.lite.TFLiteConverter.from_saved_model("tf_model")
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,
    tf.lite.OpsSet.SELECT_TF_OPS
]
converter.experimental_new_converter = True
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.allow_custom_ops = True
converter.experimental_enable_dynamic_shapes = True
tflite_model = converter.convert()

# Save the TFLite model
with open("arabic_model.tflite", "wb") as f:
    f.write(tflite_model)

In [ ]:
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, TFAutoModelForSpeechSeq2Seq
import tensorflow as tf
import numpy as np

# Load the model and processor
model_path = "/content/my_data"  # Replace with your local model directory
processor = AutoProcessor.from_pretrained(model_path)
model = AutoModelForSpeechSeq2Seq.from_pretrained(model_path)

# Convert to TensorFlow
tf_model = TFAutoModelForSpeechSeq2Seq.from_pretrained(model_path, from_pt=True)

# Create a concrete function with dynamic shapes for decoder_input_ids
@tf.function(input_signature=[
    tf.TensorSpec([1, 80, 3000], tf.float32, name="input_features"),
    tf.TensorSpec([1, None], tf.int32, name="decoder_input_ids"),
])
def serving_fn(input_features, decoder_input_ids):
    return tf_model(
        input_features=input_features,
        decoder_input_ids=decoder_input_ids,
        training=False
    )

# Save the model with the signature
tf.saved_model.save(tf_model, "tf_modell", signatures={"serving_default": serving_fn})

# Convert to TFLite
converter = tf.lite.TFLiteConverter.from_saved_model("tf_model")
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,
    tf.lite.OpsSet.SELECT_TF_OPS
]
converter.experimental_new_converter = True
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.allow_custom_ops = True
converter.experimental_enable_dynamic_shapes = True
tflite_model = converter.convert()

# Save the TFLite model
with open("arabic_model.tflite", "wb") as f:
    f.write(tflite_model)

All PyTorch model weights were used when initializing TFWhisperForConditionalGeneration.

All the weights of TFWhisperForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFWhisperForConditionalGeneration for predictions without further training.


In [ ]:
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, TFAutoModelForSpeechSeq2Seq
import tensorflow as tf
import numpy as np

# Load the model and processor
model_path = "/content/my_data"  # Replace with your local model directory
processor = AutoProcessor.from_pretrained(model_path)
model = AutoModelForSpeechSeq2Seq.from_pretrained(model_path)

# Convert to TensorFlow
tf_model = TFAutoModelForSpeechSeq2Seq.from_pretrained(model_path, from_pt=True)

# Create a concrete function with the correct input shape for Whisper
@tf.function(input_signature=[
    tf.TensorSpec([1, 80, 3000], tf.float32, name="input_features"),
    tf.TensorSpec([1, None], tf.int32, name="decoder_input_ids"),
])
def serving_fn(input_features, decoder_input_ids):
    return tf_model(
        input_features=input_features,
        decoder_input_ids=decoder_input_ids,
        training=False
    )

# Save the model with the signature
tf.saved_model.save(tf_model, "tf_model", signatures={"serving_default": serving_fn})

# Convert to TFLite with the correct shape
converter = tf.lite.TFLiteConverter.from_saved_model("tf_model")
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,
    tf.lite.OpsSet.SELECT_TF_OPS
]
converter.experimental_new_converter = True
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.allow_custom_ops = True
converter.experimental_enable_dynamic_shapes = True

# Explicitly set input shapes to ensure correctness
converter.inference_input_type = tf.float32
converter.inference_output_type = tf.float32
tflite_model = converter.convert()

# Save the TFLite model
with open("arabic_model.tflite", "wb") as f:
    f.write(tflite_model)

# Verify the TFLite model shapes
interpreter = tflite.Interpreter(model_path="arabic_model.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("All Input Details:")
for i, detail in enumerate(input_details):
    print(f"Input {i}:")
    print(f"  Name: {detail['name']}")
    print(f"  Shape: {detail['shape']}")
    print(f"  Type: {detail['dtype']}")

print("\nOutput Details:")
for detail in output_details:
    print(f"  Shape: {detail['shape']}")
    print(f"  Type: {detail['dtype']}")

All PyTorch model weights were used when initializing TFWhisperForConditionalGeneration.

All the weights of TFWhisperForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFWhisperForConditionalGeneration for predictions without further training.


All Input Details:
Input 0:
  Name: serving_default_decoder_input_ids:0
  Shape: [1 1]
  Type: <class 'numpy.int32'>
Input 1:
  Name: serving_default_input_features:0
  Shape: [   1   80 3000]
  Type: <class 'numpy.float32'>

Output Details:
  Shape: [    1     1 51865]
  Type: <class 'numpy.float32'>
  Shape: [   1 1500  512]
  Type: <class 'numpy.float32'>


In [ ]:
import tensorflow.lite as tflite
import numpy as np
import librosa
from transformers import AutoProcessor

model_path = "/content/my_data"
processor = AutoProcessor.from_pretrained(model_path)

audio_path = "/content/001 Al-Fatihah alfath.wav"
audio, sr = librosa.load(audio_path, sr=16000, mono=True)

max_length = 30 * 16000
if len(audio) > max_length:
    audio = audio[:max_length]
else:
    audio = np.pad(audio, (0, max_length - len(audio)), mode="constant")

audio = audio / np.max(np.abs(audio))
inputs = processor(audio, sampling_rate=16000, return_tensors="np")
audio_input = inputs["input_features"]  # Shape: [1, 80, 3000]

interpreter = tflite.Interpreter(model_path="arabic_model.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("All Input Details:")
for i, detail in enumerate(input_details):
    print(f"Input {i}:")
    print(f"  Name: {detail['name']}")
    print(f"  Shape: {detail['shape']}")
    print(f"  Type: {detail['dtype']}")

print("\nOutput Details:")
for detail in output_details:
    print(f"  Shape: {detail['shape']}")
    print(f"  Type: {detail['dtype']}")

decoder_input_ids = np.array([[50258]], dtype=np.int32)  # Explicitly set to INT32
transcription_ids = []

max_output_length = 448
for step in range(max_output_length):
    current_length = decoder_input_ids.shape[1]

    for i, detail in enumerate(input_details):
        if "decoder_input_ids" in detail["name"]:
            interpreter.resize_tensor_input(i, [1, current_length])
    interpreter.allocate_tensors()

    # Debug: Print the type of decoder_input_ids
    print(f"Setting decoder_input_ids with shape: {decoder_input_ids.shape}, type: {decoder_input_ids.dtype}")

    for detail in input_details:
        if "input_features" in detail["name"]:
            interpreter.set_tensor(detail["index"], audio_input)
        elif "decoder_input_ids" in detail["name"]:
            interpreter.set_tensor(detail["index"], decoder_input_ids.astype(np.int32))  # Explicit cast

    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]["index"])

    predicted_token = np.argmax(output_data[0, -1, :])
    transcription_ids.append(predicted_token)

    if predicted_token == 50256:
        break

    decoder_input_ids = np.append(decoder_input_ids, [[predicted_token]], dtype=np.int32)

transcription = processor.batch_decode([transcription_ids], skip_special_tokens=True)[0]
print("Arabic Transcription:", transcription)

All Input Details:
Input 0:
  Name: serving_default_decoder_input_ids:0
  Shape: [1 1]
  Type: <class 'numpy.int32'>
Input 1:
  Name: serving_default_input_features:0
  Shape: [   1   80 3000]
  Type: <class 'numpy.float32'>

Output Details:
  Shape: [    1     1 51865]
  Type: <class 'numpy.float32'>
  Shape: [   1 1500  512]
  Type: <class 'numpy.float32'>
Setting decoder_input_ids with shape: (1, 1), type: int32


TypeError: append() got an unexpected keyword argument 'dtype'

In [ ]:
import tensorflow.lite as tflite
import numpy as np
import librosa
from transformers import AutoProcessor

model_path = "/content/my_data"
processor = AutoProcessor.from_pretrained(model_path)

audio_path = "/content/001 Al-Fatihah alfath.wav"
audio, sr = librosa.load(audio_path, sr=16000, mono=True)

max_length = 30 * 16000
if len(audio) > max_length:
    audio = audio[:max_length]
else:
    audio = np.pad(audio, (0, max_length - len(audio)), mode="constant")

audio = audio / np.max(np.abs(audio))
inputs = processor(audio, sampling_rate=16000, return_tensors="np")
audio_input = inputs["input_features"]  # Shape: [1, 80, 3000]

interpreter = tflite.Interpreter(model_path="/content/arabic_model.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("All Input Details:")
for i, detail in enumerate(input_details):
    print(f"Input {i}:")
    print(f"  Name: {detail['name']}")
    print(f"  Shape: {detail['shape']}")
    print(f"  Type: {detail['dtype']}")

print("\nOutput Details:")
for detail in output_details:
    print(f"  Shape: {detail['shape']}")
    print(f"  Type: {detail['dtype']}")

decoder_input_ids = np.array([[50258]], dtype=np.int32)
transcription_ids = []

max_output_length = 448
for step in range(max_output_length):
    current_length = decoder_input_ids.shape[1]

    for i, detail in enumerate(input_details):
        if "decoder_input_ids" in detail["name"]:
            interpreter.resize_tensor_input(i, [1, current_length])
    interpreter.allocate_tensors()

    print(f"Setting decoder_input_ids with shape: {decoder_input_ids.shape}, type: {decoder_input_ids.dtype}")

    for detail in input_details:
        if "input_features" in detail["name"]:
            interpreter.set_tensor(detail["index"], audio_input)
        elif "decoder_input_ids" in detail["name"]:
            interpreter.set_tensor(detail["index"], decoder_input_ids)

    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]["index"])

    predicted_token = np.argmax(output_data[0, -1, :])
    transcription_ids.append(predicted_token)

    if predicted_token == 50256:
        break

    # Fix: Convert predicted_token to np.int32 and append correctly
    new_token = np.array([[predicted_token]], dtype=np.int32)
    decoder_input_ids = np.concatenate((decoder_input_ids, new_token), axis=1)

transcription = processor.batch_decode([transcription_ids], skip_special_tokens=True)[0]
print("Arabic Transcription:", transcription)

All Input Details:
Input 0:
  Name: serving_default_decoder_input_ids:0
  Shape: [1 1]
  Type: <class 'numpy.int32'>
Input 1:
  Name: serving_default_input_features:0
  Shape: [   1   80 3000]
  Type: <class 'numpy.float32'>

Output Details:
  Shape: [    1     1 51865]
  Type: <class 'numpy.float32'>
  Shape: [   1 1500  512]
  Type: <class 'numpy.float32'>
Setting decoder_input_ids with shape: (1, 1), type: int32
Setting decoder_input_ids with shape: (1, 2), type: int32
Setting decoder_input_ids with shape: (1, 3), type: int32
Setting decoder_input_ids with shape: (1, 4), type: int32
Setting decoder_input_ids with shape: (1, 5), type: int32
Setting decoder_input_ids with shape: (1, 6), type: int32
Setting decoder_input_ids with shape: (1, 7), type: int32
Setting decoder_input_ids with shape: (1, 8), type: int32
Setting decoder_input_ids with shape: (1, 9), type: int32
Setting decoder_input_ids with shape: (1, 10), type: int32
Setting decoder_input_ids with shape: (1, 11), type: int32